In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import pickle

warnings.filterwarnings("ignore")

In [2]:
# the stock data
df = pd.read_csv('data/return_predictability_data.csv')

# The industry dataframe
# ind=pd.read_csv('data/industry.csv')

with open('return_predictability.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:

ff_research = pd.read_csv('data/ff_factors.csv')
ff_research.head()

In [3]:
df['Date'] = [''.join(x.split('-')[0:2]) for x in df.DATE]

In [ ]:
df = df.merge(ff_research[['Date','macro_smb', 'macro_hml', 'macro_mkt-rf']], on = 'Date', how ='inner' )

In [14]:
from sklearn.model_selection import train_test_split

df_copy = df.copy()

train, test = train_test_split(df_copy, test_size=0.2, random_state=42)

train.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)


In [15]:
train.drop(columns=['DATE','Date','permno', 'age', 'macro_smb', 'macro_hml', 'macro_mkt-rf'])
test.drop(columns=['DATE','Date','permno', 'age', 'macro_smb', 'macro_hml', 'macro_mkt-rf'])

,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,mom12m,...,name,risk_premium,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar
1865711,4.365200e+04,1.899946,3.609794,0.124084,8.907071,0.090556,0.041042,0.000000,-0.179487,-0.428571,...,THERAGENICS CORP,-7.2750,3.544257,2.869588,0.308417,0.033996,0.0414,-0.0514,0.0068,0.000797
1545207,2.836100e+04,0.389881,0.152007,0.804455,11.519841,0.088913,-0.088367,-0.102273,0.725447,0.284639,...,KENTUCKY MEDICAL INSURANCE CO,-19.4153,3.310141,2.704522,0.488074,0.001439,0.0745,-0.1164,0.0100,0.005828
657659,3.304000e+03,2.852220,8.135156,-0.777778,NaN,0.117569,0.260231,-0.200000,0.250000,1.222222,...,MACKEY INTERNATIONAL INC,-18.9200,2.941053,2.102679,1.036731,0.028814,0.0644,-0.0724,0.0076,0.000908
3556173,7.239478e+05,1.327249,1.761589,-0.138926,13.860675,0.040723,0.413614,0.168981,0.099237,0.565217,...,INVESTMENT TECHNOLOGY GP INC NEW,2.1482,3.940750,2.920016,0.348011,0.016623,0.0003,0.0178,0.0066,0.001363
3497052,6.081792e+04,1.515756,2.297516,0.121612,9.959343,0.051836,0.085156,0.010526,0.043956,-0.025641,...,L J INTERNATIONAL INC,2.5642,3.903879,2.898462,0.333833,-0.004705,0.0004,-0.0633,0.0084,0.001016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2329786,6.992454e+05,NaN,NaN,NaN,13.795772,NaN,-0.066838,0.126147,-0.289902,NaN,...,VERISIGN INC,26.3336,4.275666,3.426020,0.174847,0.022004,0.0441,-0.0344,0.0093,0.001937
850511,4.537308e+05,1.064682,1.133547,0.078254,13.758827,0.052013,0.705023,0.116466,0.680660,2.022069,...,DILLINGHAM CORP,-17.9777,3.038287,2.188138,0.936224,0.028845,0.1630,-0.1008,0.0163,0.000922
1411750,5.343750e+03,2.279662,5.196857,-0.241548,6.240276,0.192953,0.122210,0.055556,0.058824,0.439148,...,SAN CARLOS MILLING INC,-13.3632,3.349346,2.459031,0.465284,-0.021406,0.0807,-0.0697,0.0108,0.000628
1548668,2.938200e+04,0.591770,0.350192,-0.068631,8.604746,0.024798,-0.186918,-0.036757,0.108219,0.259065,...,BOWL AMERICA INC,-18.6894,3.310141,2.704522,0.488074,0.001439,0.0745,-0.1164,0.0100,0.005828


In [18]:
train = train[~np.isnan(train['bm'])]
test = test[~np.isnan(test['bm'])]
